In [ ]:
pip install git+https://github.com/Program-Trace-Optimisation/PTO.git

# OneMax in PTO

This is a minimal example. We import `run` and `rnd`, and our `generator` is written to make random decisions using `rnd`. Our objective is just `sum`. The `max` function tells PTO that we are maximising, ie larger values of the objective are better. The `run` function is then the single-function API.

In [1]:
from pto import run, rnd
def generator(): return [rnd.choice([0, 1]) for i in range(10)]
(pheno, geno), fx = run(generator, sum, better=max)

In [ ]:
print(f'Genotype {geno}')
print(f'Solution {pheno}')
print(f'Fitness {fx}')

# Sphere in PTO

Another basic example, this time minimising the sum of squares. Here, the `generator` has an argument `N` giving the problem size. We can pass such an argument to `run` using `gen_args`. This example also demonstrates we can add a `callback` in `run`.

In [ ]:
from pto import run, rnd
N = 10
def generator(N): # standard uniform initialisation 
    return [rnd.uniform(-1, 1) for i in range(N)]
def fitness(vector): return sum([x**2 for x in vector])   

(pheno, geno), fx = run(generator, fitness, 
                        gen_args=(N,), 
                        callback=lambda x: print(f"Hello from Solver callback! {x}"),
                        better=min)

In [ ]:
print(f'Genotype {geno}')
print(f'Solution {pheno}')
print(f'Fitness {fx}')

# Symbolic Regression in PTO

This example shows how to pass arguments also to the objective function.

Symbolic regression is an interesting problem for PTO, because the promise of PTO is that it automatically implicitly defines operators suitable for the problem, given the `generator`. And indeed, the implicit operators here will resemble known mutation and homologous tree crossover operators - see our papers for details.

In [5]:
from pto import run, rnd
import random # for generating problem data

    
#################
# INSTANCE DATA #
#################
         
n_vars = 3 # problem size (number of input variables)
        
func_set = [('and', 2), ('or', 2), ('not', 1)] # functions set
term_set = ['x1', 'x2', 'x3'] # terminals set
        
target = lambda x1, x2, x3: x1 or x2 or not x3 # target function
    
# create training set
n = 10 # training set size
X_train = [[random.choice([0, 1]) for _ in range(3)] for _ in range(n)] # training inputs
y_train = [target(*xi) for xi in X_train] # training outputs

better = min
        

######################
# SOLUTION GENERATOR #
######################

# generate random expression
def generator(func_set, term_set): 

    def rnd_expr(): # Growth Initialisation
        if rnd.random() < len(term_set)/(len(term_set)+len(func_set)):
            expr = rnd.choice(term_set)
        else:
            func, arity = rnd.choice(func_set)
            if arity == 1:
                expr = '(%s %s)' % (func, rnd_expr())
            else: # arity = 2
                expr = '(%s %s %s)' % (rnd_expr(), func, rnd_expr())
        return expr
        
    return rnd_expr()


####################
# FITNESS FUNCTION #
####################

# fitness
def fitness(expr, X, y):
    
    # string to function
    f = eval("lambda x1, x2, x3 : " + expr)
    
    # predictions on traning set
    yhat = [f(*xi) for xi in X] 
    
    # error on traing set
    err = sum(abs(yhati - yi) for (yhati, yi) in zip(yhat, y))
    
    return err



(pheno, geno), fx = run(generator, fitness, 
                        gen_args=(func_set, term_set), 
                        fit_args=(X_train, y_train), better=better)


In [ ]:
print(f'Genotype {geno}')
print(f'Solution {pheno}')
print(f'Fitness {fx}')